In [5]:
import os
import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Conv1D, Dense, Embedding, Flatten, Input, LSTM, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [6]:
#
# Configuration
#
MAX_NB_WORDS=25000
MAX_SEQUENCE_LENGTH=1000
N_GLOVE_TOKENS=400000
EMBEDDING_DIM = 100

In [7]:
#
# Load the data
#
positive_dir = "aclImdb/train/pos"
negative_dir = "aclImdb/train/neg"

def read_text(filename):
        with open(filename,encoding='utf8') as f:
                return f.read().lower()

print ("Reading negative reviews.")
negative_text = [read_text(os.path.join(negative_dir, filename))
        for filename in tqdm.tqdm(os.listdir(negative_dir))]
        
print ("Reading positive reviews.")
positive_text = [read_text(os.path.join(positive_dir, filename))
        for filename in tqdm.tqdm(os.listdir(positive_dir))]


labels_index = { "negative": 0, "positive": 1 }

labels = [0 for _ in range(len(negative_text))] + \
        [1 for _ in range(len(negative_text))]
    
texts = negative_text + positive_text
 


tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np_utils.to_categorical(np.asarray(labels))
print ("data.shape = {0}, labels.shape = {1}".format(data.shape, labels.shape))

x_train, x_test, y_train, y_test = train_test_split(data, labels)


#
# Load word embeddings
#
print("Loading word embeddings.")
embeddings_index = dict()
with open("glove.6B.100d.txt",encoding='utf8') as f:
        for line in tqdm.tqdm(f, total=N_GLOVE_TOKENS):
                values = line.split()
                word, coefficients = values[0], np.asarray(values[1:], dtype=np.float32)
                embeddings_index[word] = coefficients

embedding_matrix = np.zeros((len(word_index)+1, EMBEDDING_DIM))
for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

print ("embedding_matrix.shape = {0}".format(embedding_matrix.shape))

embedding_layer = Embedding(len(word_index)+1,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)

Reading negative reviews.


100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [01:12<00:00, 173.37it/s]


Reading positive reviews.


100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [01:11<00:00, 175.90it/s]


data.shape = (25000, 1000), labels.shape = (25000, 2)
Loading word embeddings.


100%|███████████████████████████████████████████████████████████████████████| 400000/400000 [00:14<00:00, 28484.99it/s]


embedding_matrix.shape = (88583, 100)


In [8]:
#
# Build 1D ConvNet
#
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences = embedding_layer(sequence_input)


x = Conv1D(128, 5, activation="relu")(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation="relu")(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation="relu")(x)


#x = LSTM(64, dropout_W=0.2, dropout_U=0.2)(x)
#x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation="relu")(x)

preds = Dense(len(labels_index), activation="softmax")(x)

model = Model(sequence_input, preds)
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["acc"])

#
# Train the model
#
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          nb_epoch=4, batch_size=128)

C:\Users\prshk\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 18750 samples, validate on 6250 samples
Epoch 1/4
18750/18750 [==============================] - ETA: 4:06 - loss: 0.7055 - acc: 0.437 - ETA: 3:53 - loss: 0.8292 - acc: 0.519 - ETA: 3:50 - loss: 0.7862 - acc: 0.523 - ETA: 3:46 - loss: 0.7642 - acc: 0.531 - ETA: 3:44 - loss: 0.7579 - acc: 0.515 - ETA: 3:42 - loss: 0.7481 - acc: 0.523 - ETA: 3:41 - loss: 0.7416 - acc: 0.519 - ETA: 3:39 - loss: 0.7384 - acc: 0.515 - ETA: 3:37 - loss: 0.7344 - acc: 0.510 - ETA: 3:35 - loss: 0.7310 - acc: 0.507 - ETA: 3:33 - loss: 0.7275 - acc: 0.508 - ETA: 3:32 - loss: 0.7245 - acc: 0.511 - ETA: 3:30 - loss: 0.7227 - acc: 0.508 - ETA: 3:28 - loss: 0.7203 - acc: 0.508 - ETA: 3:26 - loss: 0.7190 - acc: 0.506 - ETA: 3:27 - loss: 0.7177 - acc: 0.503 - ETA: 3:27 - loss: 0.7161 - acc: 0.502 - ETA: 3:27 - loss: 0.7148 - acc: 0.500 - ETA: 3:26 - loss: 0.7138 - acc: 0.501 - ETA: 3:24 - loss: 0.7139 - acc: 0.498 - ETA: 3:23 - loss: 0.7131 - acc: 0.497 - ETA: 3:22 - loss: 0.7123 - acc: 0.498 - ETA: 3:20 - lo

18750/18750 [==============================] - ETA: 4:42 - loss: 0.4824 - acc: 0.789 - ETA: 4:52 - loss: 0.4835 - acc: 0.773 - ETA: 5:06 - loss: 0.4699 - acc: 0.789 - ETA: 4:58 - loss: 0.4287 - acc: 0.814 - ETA: 4:49 - loss: 0.4266 - acc: 0.807 - ETA: 4:44 - loss: 0.4157 - acc: 0.809 - ETA: 4:45 - loss: 0.4247 - acc: 0.805 - ETA: 4:50 - loss: 0.4415 - acc: 0.794 - ETA: 4:47 - loss: 0.4366 - acc: 0.800 - ETA: 4:45 - loss: 0.4418 - acc: 0.798 - ETA: 4:52 - loss: 0.4343 - acc: 0.802 - ETA: 4:54 - loss: 0.4341 - acc: 0.800 - ETA: 4:51 - loss: 0.4336 - acc: 0.800 - ETA: 4:57 - loss: 0.4327 - acc: 0.800 - ETA: 4:58 - loss: 0.4349 - acc: 0.797 - ETA: 4:54 - loss: 0.4353 - acc: 0.797 - ETA: 4:49 - loss: 0.4356 - acc: 0.797 - ETA: 4:45 - loss: 0.4289 - acc: 0.803 - ETA: 4:40 - loss: 0.4301 - acc: 0.803 - ETA: 4:36 - loss: 0.4276 - acc: 0.807 - ETA: 4:32 - loss: 0.4285 - acc: 0.806 - ETA: 4:28 - loss: 0.4279 - acc: 0.805 - ETA: 4:26 - loss: 0.4261 - acc: 0.807 - ETA: 4:22 - loss: 0.4237 - acc: 0

18750/18750 [==============================] - ETA: 4:59 - loss: 0.3793 - acc: 0.820 - ETA: 4:52 - loss: 0.3471 - acc: 0.847 - ETA: 4:46 - loss: 0.3361 - acc: 0.854 - ETA: 4:45 - loss: 0.3107 - acc: 0.873 - ETA: 4:42 - loss: 0.3118 - acc: 0.870 - ETA: 4:43 - loss: 0.3175 - acc: 0.862 - ETA: 4:43 - loss: 0.3142 - acc: 0.863 - ETA: 4:41 - loss: 0.3116 - acc: 0.861 - ETA: 4:40 - loss: 0.3076 - acc: 0.866 - ETA: 4:54 - loss: 0.3053 - acc: 0.868 - ETA: 5:06 - loss: 0.3088 - acc: 0.869 - ETA: 5:05 - loss: 0.3100 - acc: 0.867 - ETA: 5:06 - loss: 0.3104 - acc: 0.868 - ETA: 5:04 - loss: 0.3068 - acc: 0.870 - ETA: 4:58 - loss: 0.3082 - acc: 0.872 - ETA: 4:53 - loss: 0.3059 - acc: 0.872 - ETA: 4:48 - loss: 0.3087 - acc: 0.870 - ETA: 4:43 - loss: 0.3046 - acc: 0.872 - ETA: 4:39 - loss: 0.3071 - acc: 0.871 - ETA: 4:35 - loss: 0.3089 - acc: 0.871 - ETA: 4:31 - loss: 0.3112 - acc: 0.870 - ETA: 4:27 - loss: 0.3145 - acc: 0.870 - ETA: 4:24 - loss: 0.3202 - acc: 0.866 - ETA: 4:20 - loss: 0.3188 - acc: 0

18750/18750 [==============================] - ETA: 4:54 - loss: 0.2689 - acc: 0.937 - ETA: 4:48 - loss: 0.2600 - acc: 0.910 - ETA: 4:45 - loss: 0.2521 - acc: 0.914 - ETA: 4:43 - loss: 0.2438 - acc: 0.914 - ETA: 4:42 - loss: 0.2327 - acc: 0.915 - ETA: 4:40 - loss: 0.2343 - acc: 0.915 - ETA: 4:38 - loss: 0.2351 - acc: 0.912 - ETA: 4:38 - loss: 0.2305 - acc: 0.913 - ETA: 4:39 - loss: 0.2290 - acc: 0.913 - ETA: 4:35 - loss: 0.2318 - acc: 0.911 - ETA: 4:32 - loss: 0.2317 - acc: 0.909 - ETA: 4:29 - loss: 0.2337 - acc: 0.907 - ETA: 4:27 - loss: 0.2376 - acc: 0.904 - ETA: 4:24 - loss: 0.2456 - acc: 0.901 - ETA: 4:21 - loss: 0.2487 - acc: 0.899 - ETA: 4:18 - loss: 0.2519 - acc: 0.898 - ETA: 4:15 - loss: 0.2476 - acc: 0.902 - ETA: 4:12 - loss: 0.2472 - acc: 0.901 - ETA: 4:10 - loss: 0.2532 - acc: 0.898 - ETA: 4:08 - loss: 0.2570 - acc: 0.896 - ETA: 4:05 - loss: 0.2584 - acc: 0.896 - ETA: 4:03 - loss: 0.2602 - acc: 0.894 - ETA: 4:00 - loss: 0.2604 - acc: 0.895 - ETA: 3:59 - loss: 0.2620 - acc: 0

In [9]:
%store -r data
data.head()

,sentiment,text
1,positive,Greetings and welcome to the Microsoft Fiscal ...
10,positive,technology. Microsoft 365 helps every organiza...
100,positive,is coming from the line of Walter Pritchard wi...
101,neutral,we're growing at eye-popping rates right now. ...
102,positive,the key things that we think about is differen...


In [10]:
texts = data.text
labels = data.sentiment.map(str.lower).map({'positive' : 1, 'neutral' : 0, 'negative' : 0}).map(int)

In [11]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index


test_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
labels_pred = model.predict(test_data)
labels_pred

array([[ 0.77918488,  0.22081508],
       [ 0.93208724,  0.06791282],
       [ 0.89135247,  0.10864758],
       ..., 
       [ 0.83458674,  0.16541333],
       [ 0.78091168,  0.21908833],
       [ 0.82761705,  0.17238298]], dtype=float32)

In [13]:
l_p = np.argmax(labels_pred, axis=1)

In [14]:
confusion_matrix(y_pred=l_p, y_true=labels)

array([[119, 186],
       [120, 197]], dtype=int64)

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8858300   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048     
__________